In [1]:

import torch
import torch.nn as nn


class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.encoder = nn.Sequential(
            # [3, 32, 32]
            nn.Conv2d(3, 64, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            # [64, 16, 16]
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            # [128, 8, 8]
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2)
            # [256, 4, 4] -> latent shape: 4096
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 5, stride=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 9, stride=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 17, stride=1),
            nn.Tanh()
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x = self.decoder(x1)
        return x1, x

In [2]:
import numpy as np
from torch.utils.data import Dataset


def preprocess(image_list):
    '''
    Normalize Image and Permute (N,H,W,C) to (N,C,W,H)
    :param image_list: list of images (9000, 32, 32, 3)
    :return: image_list: list of images (9000, 3, 32, 32)
    '''
    image_list = np.array(image_list)
    image_list = np.transpose(image_list, (0, 3, 1, 2))
    image_list = (image_list / 255.0) * 2 - 1
    image_list = image_list.astype(np.float32)
    return image_list


class ImgDataSet(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)

In [9]:
import matplotlib.pyplot as plt
import torch
import random
import numpy as np


def save_model(model, dir, args):
    if args['n_gpu'] > 1:
        torch.save(model.state_dict(), dir+'.pth')
    model = model.to(torch.device('cpu'))
    torch.save(model, dir+'-cpu.pth')
    print('-' * 20)
    print('Save Success!')


def count_parameters(model, only_trainable=False):
    '''
    count the number of parameters need to train
    :param model:
    :param only_trainable:
    :return:
    '''
    if only_trainable:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


def same_seeds(seed):
    '''
    set the same seed to reproduce the result
    :param seed:
    :return:
    '''
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def cal_acc(gt, pred):
    '''
    compute categorization accuracy of our task
    :param gt:  ground truth labels (9000, )
    :param pred: predicted labels (9000, )
    :return:
    '''
    correct = np.sum(gt == pred)
    acc = correct / gt.shape[0]
    return max(acc, 1-acc)


def plot_scatter(feat, label, savefig=None):
    '''
    plot scatter image
    :param feat: the (x, y) coordinate of clustering result, shape (9000, 2)
    :param label: ground truth label of image (0/1), shape (9000, 2)
    :param savefig:
    :return:
    '''
    X = feat[:, 0]
    Y = feat[:, 1]
    plt.scatter(X, Y, c=label)
    plt.legend(loc='best')
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()
    return

In [12]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


def train_loop(autoencoder, dataloader, args: dict):
    device = args['device']
    n_gpu = args['n_gpu']
    epochs = args['epochs']
    criterion = nn.MSELoss()
    learning_rate = 1e-5
    decay = 1e-5
    opt = torch.optim.Adam(autoencoder.parameters(),
                           lr=learning_rate,
                           weight_decay=decay)

    autoencoder.train()
    mse_loss = 0
    losses = []
    for epoch in range(epochs):
        for batch in dataloader:
            batch = batch.to(device)
            hidden, out = autoencoder(batch)
            loss = criterion(out, batch)

            opt.zero_grad()
            loss.backward()
            opt.step()

            if n_gpu > 1:
                loss = loss.mean()
            mse_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            losses.append(mse_loss)
            mse_loss = 0
        print("epoch %d, mse loss: %.2f" % (epoch, mse_loss))

    return losses


def eval(autoencoder, device, samples, dataset):
    autoencoder = autoencoder.to(device)
    autoencoder.eval()
    dataloader = DataLoader(dataset,
                            batch_size=1,
                            shuffle=True)
    inputs = []
    outputs = []
    with torch.no_grad():
        for sample in range(samples):
            input = next(iter(dataloader))
            input = input.to(device)
            hidden, output = autoencoder(input)
            input = input[0]
            input = np.transpose(input, (1, 2, 0))
            inputs.append(input)
            output = output[0]
            output = np.transpose(output, (1, 2, 0))
            outputs.append(output)

        return inputs, outputs

In [14]:
import numpy as np

print('Loading data...')
train_data = np.load('./train.npy')
test_data = np.load('./test.npy')
train_data, test_data = preprocess(train_data), preprocess(test_data)
dataset = ImgDataSet(train_data)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
print('Loading model...')
model = AE()
print('Setting cuda&cpu...')
device = torch.device('cpu')
n_gpu = 0
if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    gpu_ids = list(range(0, n_gpu))
    if n_gpu > 1:
        model = torch.nn.DataParallel(
            model, device_ids=gpu_ids, output_device=gpu_ids[-1]
        )
        print('-> GPU training available! Training will use GPU(s) {}\n'.format(gpu_ids))
    device = torch.device('cuda')
print('device: ', device)

model = model.to(device)
args = {'epochs': 10, 'device': device, 'n_gpu': n_gpu}
save_model(model, 'AE', args)
losses = train_loop(model, dataloader, args)

np.save(losses, 'losses.npy')
save_model(model, 'AE', args)

Loading data...
Loading model...
Setting cuda&cpu...
device:  cpu


KeyboardInterrupt: 